<a href="https://colab.research.google.com/github/selenaxu-hub/cf_ai_agent/blob/main/hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 26.2 MB/s eta 0:00:00


In [3]:
#import packages
import time
from copy import deepcopy
import json
import pymongo
import os
import datetime
import pandas
import math

In [4]:
#get the username and and password information - You need to run it one time at start
from getpass import getpass

USER = "ubuntu"
PASS = getpass("Enter MongoDB password: ")
HOST = "3.89.49.127"
PORT = 27017

Enter MongoDB password: ··········


In [5]:
#set up mongo connection
mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(USER,
                                                         PASS,
                                                         HOST,
                                                         PORT,
                                                         "lahman")

client = pymongo.MongoClient(mongo_url)

In [6]:
# Testing connection!
db = client['lahman']
#now print all collection names
print("collection names: {}".format(db.list_collection_names()))
reading = db['Batting'].find_one()
#print one batting documents
reading["_id"] = str(reading["_id"])
print(reading)

collection names: ['SeriesPost', 'AwardsShareManagers', 'PitchingPost', 'FieldingPost', 'ManagersHalf', 'Salaries', 'Pitching', 'AwardsSharePlayers', 'FieldingOF', 'AllstarFull', 'Schools', 'BattingPost', 'AwardsManagers', 'HallOfFame', 'Batting', 'TeamsFranchises', 'TeamsHalf', 'Appearances', 'Fielding', 'SchoolsPlayers', 'Master', 'Teams', 'Managers', 'AwardsPlayers']
{'_id': '698aaad02b5ba8139a89e4fe', 'playerID': 'aardsda01', 'yearID': 2004, 'stint': 1, 'teamID': 'SFN', 'lgID': 'NL', 'G': 11, 'G_batting': 11, 'AB': 0, 'R': 0, 'H': 0, '2B': 0, '3B': 0, 'HR': 0, 'RBI': 0, 'SB': 0, 'CS': 0, 'BB': 0, 'SO': 0, 'IBB': 0, 'HBP': 0, 'SH': 0, 'SF': 0, 'GIDP': 0, 'G_old': 11}


## Part 1 Queries

1. The number of all stars in allstarfull.
2. The most home runs in a season by a single player (using the batting table).
3. The playerid of the player with the most home runs in a season.
4. The number of leagues in the batting table.
5. Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. Note you will nead to cast hits to get a decimal: cast(h as real)
6. The teamid with the fewest hits in the year 2000 (ie., yearid = '2000'). Return both the teamid, and the number of hits. Note you can use ORDER BY column and LIMIT 1.
7. The teamid in the year 2000 (i.e., yearid = '2000') with the highest average batting average. Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.
8. The number of all stars the giants (teamid = 'SFN') had in 2000.
9. The yearid which the giants had the most all stars.
10. The average salary in year 2000.
11. The number of positions (e.g., catchers, pitchers) that have average salaries greather than 2000000 in yearid 2000. You will need to join fielding with salaries.
12. The number of errors Barry Bonds had in 2000.
13. The average salary of all stars in 2000.
14. The average salary of non-all stars in 2000.

In [16]:
# I used AI in this part to help me with the syntax

import math

# Q1: Total All-Stars
def q1():
    # Just counting every single entry in the AllstarFull collection
    return db.AllstarFull.count_documents({})

# Q2: Most Home Runs in a single season
def q2():
    # I'm converting HR to an int on the fly to find the true numerical max
    res = list(db.Batting.aggregate([
        {"$project": {"HR_int": {"$toInt": "$HR"}}},
        {"$sort": {"HR_int": -1}},
        {"$limit": 1}
    ]))
    return res[0]['HR_int'] if res else 0

# Q3: PlayerID for the Home Run record holder
def q3():
    # Similar to Q2, but I'm grabbing the player's ID this time
    res = list(db.Batting.aggregate([
        {"$project": {"playerID": 1, "HR_int": {"$toInt": "$HR"}}},
        {"$sort": {"HR_int": -1}},
        {"$limit": 1}
    ]))
    return res[0]['playerID'] if res else None

# Q4: Count of unique leagues
def q4():
    # Getting the list of unique lgID values and checking the length
    return len(db.Batting.distinct("lgID"))

# Q5: Barry Bonds' average batting average
def q5():
    # I calculate H/AB for each season first,
    # then take the average of those results to hit that exact decimal.
    pipeline = [
        {"$match": {"playerID": "bondsba01", "AB": {"$gt": 0, "$ne": ""}}},
        {"$project": {
            "season_ba": {"$divide": [{"$toInt": "$H"}, {"$toInt": "$AB"}]}
        }},
        {"$group": {
            "_id": None,
            "final_avg": {"$avg": "$season_ba"}
        }}
    ]
    res = list(db.Batting.aggregate(pipeline))
    return res[0]['final_avg'] if res else 0.0

# Q6: Team with fewest hits in 2000
def q6():
    # I'm filtering for the year 2000 and sorting hits in ascending order
    res = list(db.Teams.aggregate([
        {"$match": {"yearID": {"$in": [2000, "2000"]}}},
        {"$project": {"teamID": 1, "H_int": {"$toInt": "$H"}}},
        {"$sort": {"H_int": 1}},
        {"$limit": 1}
    ]))
    return (res[0]['teamID'], res[0]['H_int']) if res else None

# Q7: Team with highest avg batting average in 2000
def q7():
    # Grouping by team, calculating each player's BA, then averaging it for the team
    res = list(db.Batting.aggregate([
        {"$match": {"yearID": {"$in": [2000, "2000"]}}},
        {"$project": {"teamID": 1, "H": {"$toInt": "$H"}, "AB": {"$toInt": "$AB"}}},
        {"$match": {"AB": {"$gt": 0}}},
        {"$group": {"_id": "$teamID", "avg_ba": {"$avg": {"$divide": ["$H", "$AB"]}}}},
        {"$sort": {"avg_ba": -1}},
        {"$limit": 1}
    ]))
    return (res[0]['_id'], res[0]['avg_ba']) if res else None

# Q8: Giants All-Stars in 2000
def q8():
    # Simple filter for the team 'SFN' and the year 2000
    return db.AllstarFull.count_documents({"teamID": "SFN", "yearID": {"$in": [2000, "2000"]}})

# Q9: Year with the most Giants All-Stars
def q9():
    # I'm grouping all Giants entries by year and counting them up
    res = list(db.AllstarFull.aggregate([
        {"$match": {"teamID": "SFN"}},
        {"$group": {"_id": "$yearID", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 1}
    ]))
    return int(res[0]['_id']) if res else None

# Q10: Average salary in 2000
def q10():
    # Converting salaries to doubles to get a precise average
    res = list(db.Salaries.aggregate([
        {"$match": {"yearID": {"$in": [2000, "2000"]}}},
        {"$project": {"salary": {"$toDouble": "$salary"}}},
        {"$group": {"_id": None, "avg_sal": {"$avg": "$salary"}}}
    ]))
    return res[0]['avg_sal'] if res else 0.0

# Q11: Positions with avg salary > 2M in 2000
def q11():
    # I use a lookup to join fielding with salaries, then filter the average
    res = [
        {"$match": {"yearID": {"$in": [2000, "2000"]}}},
        {"$lookup": {
            "from": "Salaries",
            "let": {"pid": "$playerID", "yid": "$yearID"},
            "pipeline": [
                {"$match": {"$expr": {"$and": [
                    {"$eq": ["$playerID", "$$pid"]},
                    {"$eq": ["$yearID", "$$yid"]}
                ]}}}
            ],
            "as": "joined_sal"
        }},
        {"$unwind": "$joined_sal"},
        {"$group": {
            "_id": "$POS",
            "pos_avg": {"$avg": {"$toDouble": "$joined_sal.salary"}}
        }},
        {"$match": {"pos_avg": {"$gt": 2000000}}}
    ]
    return len(list(db.Fielding.aggregate(res)))

# Q12: Barry Bonds errors in 2000
def q12():
    # I'm summing up all error records for Bonds in 2000
    res = list(db.Fielding.find({"playerID": "bondsba01", "yearID": {"$in": [2000, "2000"]}}))
    return sum(int(doc['E']) for doc in res if doc.get('E'))

# Q13: Average salary of All-Stars in 2000
def q13():
    # I get the All-Star IDs first, then average their salaries and floor it
    as_ids = db.AllstarFull.distinct("playerID", {"yearID": {"$in": [2000, "2000"]}})
    res = list(db.Salaries.aggregate([
        {"$match": {"yearID": {"$in": [2000, "2000"]}, "playerID": {"$in": as_ids}}},
        {"$group": {"_id": None, "avg_sal": {"$avg": {"$toDouble": "$salary"}}}}
    ]))
    return math.floor(res[0]['avg_sal']) if res else 0

# Q14: Average salary of non-All-Stars in 2000
def q14():
    # Same as Q13, but I'm using nin to look at the other players
    as_ids = db.AllstarFull.distinct("playerID", {"yearID": {"$in": [2000, "2000"]}})
    res = list(db.Salaries.aggregate([
        {"$match": {"yearID": {"$in": [2000, "2000"]}, "playerID": {"$nin": as_ids}}},
        {"$group": {"_id": None, "avg_sal": {"$avg": {"$toDouble": "$salary"}}}}
    ]))
    return math.floor(res[0]['avg_sal']) if res else 0

In [17]:
def test_q1():
    result_q1 = q1()
    assert result_q1 == 4834

def test_q2():
    result_q2 = q2()
    assert result_q2 == 73

def test_q3():
    result_q3 = q3()
    assert result_q3 == 'bondsba01'

def test_q4():
    result_q4 = q4()
    assert result_q4 == 7

def test_q5():
    result_q5 = q5()
    assert math.isclose(result_q5, 0.29830576286005, rel_tol=1e-6)

def test_q6():
    result_q6 = q6()
    assert result_q6[0] == 'MIL'
    assert result_q6[1] == 1366

def test_q7():
    result_q7 = q7()
    assert result_q7[0] == 'SEA'
    assert math.isclose(result_q7[1], 0.280799158489005, rel_tol=1e-6)

def test_q8():
    result_q8 = q8()
    assert result_q8 == 2

def test_q9():
    result_q9 = q9()
    assert result_q9 in [1961, 1962, 1966]

def test_q10():
    result_q10 = q10()
    assert int(result_q10) == 1992984

def test_q11():
    result_q11 = q11()
    assert result_q11 == 3

def test_q12():
    result_q12 = q12()
    assert result_q12 == 6

def test_q13():
    result_q13 = q13()
    assert int(result_q13) == 5388841

def test_q14():
    result_q14 = q14()
    assert int(result_q14) == 1692309

In [18]:
test_q1()
print('Q1 - succesful')
test_q2()
print('Q2 - succesful')
test_q3()
print('Q3 - succesful')
test_q4()
print('Q4 - succesful')
test_q5()
print('Q5 - succesful')
test_q6()
print('Q6 - succesful')
test_q7()
print('Q7 - succesful')
test_q8()
print('Q8 - succesful')
test_q9()
print('Q9 - succesful')
test_q10()
print('Q10 - succesful')
test_q11()
print('Q11 - succesful')
test_q12()
print('Q12 - succesful')
test_q13()
print('Q13 - succesful')
test_q14()
print('Q14 - succesful')

print('All asserts are successful!')

Q1 - succesful
Q2 - succesful
Q3 - succesful
Q4 - succesful
Q5 - succesful
Q6 - succesful
Q7 - succesful
Q8 - succesful
Q9 - succesful
Q10 - succesful
Q11 - succesful
Q12 - succesful
Q13 - succesful
Q14 - succesful
All asserts are successful!
